<a href="https://colab.research.google.com/github/AlexShaGH/Gap-Phenomenon-in-Stock-Market/blob/main/GAPs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Nasdaq-Data-Link

In [ ]:
# import libraries
from google.colab import files
from google.colab import drive
import pandas as pd
import io
import os
# from pandas.core.indexes.datetimes import date
from datetime import datetime, date
import csv
import yfinance as yf
import nasdaqdatalink

In [ ]:
# connect G-DRIVE
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [ ]:
# set up names and pathes
GIT_PATH = "https://github.com/fja05680/sp500.git"
PRJ_PATH = "/content/gdrive/MyDrive/GAPs"
CSV_DL_PATH = "/composition/repo"
CSV_CR_PATH = "/composition/current"
DATA_PATH = "/composition/current/data"

In [ ]:
# get the latest data from repository:
# https://github.com/fja05680/sp500
!git clone "{GIT_PATH}" ./temp # clone github repository to temp folder
!mkdir -p "{PRJ_PATH}{CSV_DL_PATH}" # create a folder for downloaded files if it does mot exist
!mv ./temp/*.csv "{PRJ_PATH}{CSV_DL_PATH}" # move all CSV files to preserve
!rm -rf ./temp # clean up temp folder

Cloning into './temp'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 338 (delta 74), reused 64 (delta 49), pack-reused 243 (from 1)
Receiving objects: 100% (338/338), 1.44 MiB | 8.45 MiB/s, done.
Resolving deltas: 100% (236/236), done.


In [ ]:
# compare content of the newly downloaded repo with currently used composition:
print("Files donwloaded from repository:")
!ls -l "{PRJ_PATH}{CSV_DL_PATH}"
print("Current composition file:")
!ls -l "{PRJ_PATH}{CSV_CR_PATH}"

Files donwloaded from repository:
total 13016
-rw-------+ 1 root root    2412 Oct 16 15:55  sp500_changes_since_2019.csv
-rw-------+ 1 root root   53270 Oct 16 15:55  sp500.csv
-rw-------+ 1 root root 5452585 Oct 16 15:55 'S&P 500 Historical Components & Changes(08-17-2024).csv'
-rw-------+ 1 root root 7791584 Oct 16 15:55 'S&P 500 Historical Components & Changes.csv'
-rw-------+ 1 root root   26833 Oct 16 15:55  sp500_ticker_start_end.csv
Current composition file:
total 10719
drwx------ 2 root root    4096 Apr 17  2023  data
-rw------- 1 root root 5416748 Jun 12  2023 'S&P 500 Historical Components & Changes(04-16-2023).csv'
-rw------- 1 root root 5410428 Apr 16  2023 'S&P 500 Historical Components & Changes(12-29-2022).csv'
-rw------- 1 root root     176 Jun 11  2023 'S&P-Historical-Composition-Date-Ranges-Combined.gsheet'
-rw------- 1 root root   31139 Jun 12  2023 'S&P-Historical-Composition-Date-Ranges.csv'
-rw------- 1 root root     176 Oct 16  2023 'S&P-Historical-Composition-Da

In [ ]:
# Set the name for the newest historical composition file
# HIST_COMP_FL_NAME = 'S&P 500 Historical Components & Changes(12-29-2022).csv'
# HIST_COMP_FL_NAME = 'S&P 500 Historical Components & Changes(04-16-2023).csv'
HIST_COMP_FL_NAME = 'S&P 500 Historical Components & Changes(08-17-2024).csv'

In [ ]:
# If newer, copy it to the "current" folder.
# Otherwise this cell could be skipped.
!cp -u "{PRJ_PATH}{CSV_DL_PATH}/{HIST_COMP_FL_NAME}" "{PRJ_PATH}{CSV_CR_PATH}"

In [ ]:
# load the S&P500 historical composition
hist_comp_file_path = PRJ_PATH + CSV_CR_PATH + '/' + HIST_COMP_FL_NAME
df = pd.read_csv(hist_comp_file_path)
print(df)

            date                                            tickers
0     1996-01-02  AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...
1     1996-01-03  AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...
2     1996-01-04  AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...
3     1996-01-10  AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...
4     1996-01-11  AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD...
...          ...                                                ...
2678  2024-03-25  A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM...
2679  2024-04-03  A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM...
2680  2024-05-08  A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM...
2681  2024-06-24  A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM...
2682  2024-07-08  A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM...

[2683 rows x 2 columns]


In [ ]:
# form a set of all stocks tickers
tickers_set = set()
for composition in df['tickers']:
  curr_composition = composition.split(',')
  tickers_set.update(curr_composition)

print("Total number of tickers: {}".format(len(tickers_set)))

Total number of tickers: 1168


In [ ]:
# form a dictionary where a key is a ticker
# and a value is a list of data ranges when a stock was in the S&P index
sp500_composition = {}
counter = 1
total = len(tickers_set)
print(f"Processing {total} tickers")
for tcr in tickers_set:
  print(f"\rProcessing ticker # {counter}/{total}: {tcr}        ",end="")
  listed = False
  # declare time stamp variables
  start_timestamp = datetime.now()
  last_timestamp = datetime.now()
  # list of ranges when the current stock was a part of S&P index
  range_list = []
  # iterate through the time steps
  for timestep in df.itertuples():
    curr_timestamp = timestep.date
    curr_tickers = timestep.tickers.split(',')
    #print(curr_timestamp,curr_tickers)
    if tcr not in curr_tickers:# ticker is not in current composition
      if listed: # stock was listed before but now it is not
        # form an entry in the list containing start and stop time stamps
        range_entry = [start_timestamp,last_timestamp]
        # add it to the list of ranges
        range_list.append(range_entry)
        listed = False
    else: # ticker is in current composition
      if not listed: # stock was not listed previously
        # start tracking new range
        start_timestamp = curr_timestamp
        listed = True
    last_timestamp = curr_timestamp
  # we are done with current ticker
  if listed:  # need to check if the range needs to be closed
    # form an entry in the list containing start and stop time stamps
    range_entry = [start_timestamp,last_timestamp]
    # add it to the list of ranges
    range_list.append(range_entry)
  # add ticker and ranges to global dictionary
  sp500_composition[tcr] = range_list
  counter += 1

print('\rDone                                     ')

Processing 1168 tickers
Done                                     


In [ ]:
# create dataframe with linear ranges for each stock component
# and save it to CSV file
field_names = ['Ticker', 'Start', 'End']
csv_file_name = 'S&P-Historical-Composition-Date-Ranges.csv'
ranges_file_path = PRJ_PATH + CSV_CR_PATH + "/" + csv_file_name
df_ranges = pd.DataFrame(columns = field_names)

for ticker, date_ranges in sp500_composition.items():
  for date_range in date_ranges:
    df_ranges.loc[len(df_ranges.index)] = [ticker,date_range[0],date_range[1]]

df_ranges.to_csv(ranges_file_path, index = False)

In [ ]:
# We can pick up where we left just from here by loading all found ranges
csv_file_name = 'S&P-Historical-Composition-Date-Ranges.csv'
ranges_file_path = PRJ_PATH + CSV_CR_PATH + "/" + csv_file_name
df_ranges = pd.read_csv(ranges_file_path)
print(df_ranges.describe())
print(df_ranges)

       Ticker       Start         End
count    1216        1216        1216
unique   1168         582         575
top       COV  1996-01-02  2024-07-08
freq        3         487         503
     Ticker       Start         End
0       BWA  2011-12-19  2024-07-08
1      VIAB  2006-01-03  2019-11-21
2      ATGE  2009-06-09  2012-09-25
3       SLM  1998-07-01  2014-04-22
4      QCOM  1999-07-22  2024-07-08
...     ...         ...         ...
1211    SMI  1996-01-02  2000-12-11
1212    JHF  2001-06-29  2004-04-28
1213    JEF  2007-08-27  2019-09-23
1214   EVHC  2016-12-02  2018-10-10
1215   PAYC  2020-01-28  2024-07-08

[1216 rows x 3 columns]


In [ ]:
# iterate ranges and retrieve stocks data using Yahoo Finance services
hist_data_path_yahoo = PRJ_PATH + DATA_PATH + "/yahoo/"
if not os.path.exists(hist_data_path_yahoo):
  os.makedirs(hist_data_path_yahoo)

df_ranges_extended = df_ranges.copy()

print('Retrieving historical data')
for indx,(ticker, start, end) in df_ranges_extended.iterrows():
  stock_ticker = yf.Ticker(ticker)
  print(f'index: {indx}/{len(df_ranges_extended)}, ticker: {ticker}, start: {start}, stop: {end}')
  try:
    stock_info = stock_ticker.info
    df_ranges_extended.at[indx,'Categorized'] = True
    df_ranges_extended.at[indx,'Industry'] = stock_info['industry']
    df_ranges_extended.at[indx,'Sector'] = stock_ticker.info['sector']
    df_history = yf.download(ticker, start=start, end=end)
    if df_history.empty:
      df_ranges_extended.at[indx,'Downloaded'] = False
    else:
      df_ranges_extended.at[indx,'Downloaded'] = True
      # save the history to a file
      hist_file_name = ticker + '-' + start + '-' + end + '.csv'
      hist_file_path = hist_data_path_yahoo + hist_file_name
      df_history.to_csv(hist_file_path)
  except:
    df_ranges_extended.at[indx,'Categorized'] = False
    df_ranges_extended.at[indx,'Downloaded'] = False

csv_file_name = 'S&P-Historical-Composition-Date-Ranges-Yahoo.csv'
ext_ranges_file_path = PRJ_PATH + CSV_CR_PATH + "/" + csv_file_name
df_ranges_extended.to_csv(ext_ranges_file_path, index = False)

print('Done')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Retrieving historical data
index: 0/1216, ticker: BWA, start: 2011-12-19, stop: 2024-07-08
index: 1/1216, ticker: VIAB, start: 2006-01-03, stop: 2019-11-21
index: 2/1216, ticker: ATGE, start: 2009-06-09, stop: 2012-09-25


index: 3/1216, ticker: SLM, start: 1998-07-01, stop: 2014-04-22


[*********************100%***********************]  1 of 1 completed


index: 4/1216, ticker: QCOM, start: 1999-07-22, stop: 2024-07-08


...

index: 1208/1216, ticker: SRE, start: 1998-06-29, stop: 2024-07-08


[*********************100%***********************]  1 of 1 completed


index: 1209/1216, ticker: LOR, start: 1996-01-02, stop: 1996-04-22
index: 1210/1216, ticker: AV, start: 2000-10-02, stop: 2007-10-25
index: 1211/1216, ticker: SMI, start: 1996-01-02, stop: 2000-12-11
index: 1212/1216, ticker: JHF, start: 2001-06-29, stop: 2004-04-28
index: 1213/1216, ticker: JEF, start: 2007-08-27, stop: 2019-09-23


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


index: 1214/1216, ticker: EVHC, start: 2016-12-02, stop: 2018-10-10
index: 1215/1216, ticker: PAYC, start: 2020-01-28, stop: 2024-07-08
Done


In [ ]:
# check the quality of data
print(df_ranges_extended.describe())
print('\nIndustries:')
print(df_ranges_extended['Industry'].dropna().unique())
print('\nSectors:')
print(df_ranges_extended['Sector'].dropna().unique())

       Ticker       Start         End Categorized  \
count    1216        1216        1216        1216   
unique   1168         582         575           2   
top       COV  1996-01-02  2024-07-08        True   
freq        3         487         503         751   

                              Industry      Sector Downloaded  
count                              751         751       1216  
unique                             148          11          2  
top     Specialty Industrial Machinery  Technology       True  
freq                                23         108        654  

Industries:
['Auto Parts' 'Education & Training Services' 'Credit Services'
 'Semiconductors' 'Oil & Gas E&P' 'Luxury Goods' 'Aerospace & Defense'
 'Farm & Heavy Construction Machinery' 'Asset Management'
 'Consulting Services' 'Banks - Diversified' 'Agricultural Inputs'
 'Chemicals' 'Medical Devices' 'Insurance - Property & Casualty'
 'Software - Infrastructure' 'Electrical Equipment & Parts'
 'Rental & Leasi

In [ ]:
# iterate ranges and retrieve stocks data using NASDAQ services
# Premium dataset QUOTEMEDIA/PRICES
from google.colab import userdata
NASDAQ_API_KEY = userdata.get('NASDAQ_API_KEY')

hist_data_path_quotemedia = PRJ_PATH + DATA_PATH + "/quotemedia/"
if not os.path.exists(hist_data_path_quotemedia):
  os.makedirs(hist_data_path_quotemedia)

df_ranges_extended = df_ranges.copy()

print('Retrieving historical data')
for indx,(ticker, start, end) in df_ranges_extended.iterrows():
  print(f'index: {indx}/{len(df_ranges_extended)}, ticker: {ticker}, start: {start}, stop: {end}')
  try:
    df_history = nasdaqdatalink.get_table('QUOTEMEDIA/PRICES',
      qopts = { 'columns': ['date', 'Open',	'high',	'low',	'close',	'adj_close', 'volume'] },
      ticker = ticker,
      date = { 'gte': start, 'lte': end },
      paginate = True,
      api_key = NASDAQ_API_KEY)
    df_history.sort_values(by=['date'],inplace=True)
    df_history.reset_index(drop=True,inplace=True)

    if df_history.empty:
      df_ranges_extended.at[indx,'Downloaded'] = False
    else:
      df_ranges_extended.at[indx,'Downloaded'] = True
      # save the history to a file
      hist_file_name = ticker + '-' + start + '-' + end + '.csv'
      hist_file_path = hist_data_path_quotemedia + hist_file_name
      df_history.to_csv(hist_file_path,index = False)
  except:
    df_ranges_extended.at[indx,'Downloaded'] = False

csv_file_name = 'S&P-Historical-Composition-Date-Ranges-Quotemedia.csv'
ext_ranges_file_path = PRJ_PATH + CSV_CR_PATH + "/" + csv_file_name
df_ranges_extended.to_csv(ext_ranges_file_path, index = False)

print('Done')

3psKHZxiustdzigWya7c
Retrieving historical data
index: 0/1216, ticker: BWA, start: 2011-12-19, stop: 2024-07-08
index: 1/1216, ticker: VIAB, start: 2006-01-03, stop: 2019-11-21
index: 2/1216, ticker: ATGE, start: 2009-06-09, stop: 2012-09-25
index: 3/1216, ticker: SLM, start: 1998-07-01, stop: 2014-04-22
index: 4/1216, ticker: QCOM, start: 1999-07-22, stop: 2024-07-08
index: 5/1216, ticker: EP, start: 1999-10-26, stop: 2012-05-23
index: 6/1216, ticker: TPR, start: 2004-09-01, stop: 2024-07-08
index: 7/1216, ticker: TDG, start: 2016-06-03, stop: 2024-07-08
index: 8/1216, ticker: GP, start: 1996-01-02, stop: 2005-12-16
index: 9/1216, ticker: SII, start: 2006-10-02, stop: 2010-08-19
index: 10/1216, ticker: EFX, start: 1997-06-19, stop: 2024-07-08
index: 11/1216, ticker: ANDW, start: 1996-01-02, stop: 2006-09-28
index: 12/1216, ticker: FLIR, start: 2009-01-02, stop: 2021-04-20
index: 13/1216, ticker: BAC, start: 1996-01-02, stop: 2024-07-08
index: 14/1216, ticker: USBC, start: 1996-01-02, 

In [ ]:
df_ranges_extended.describe()

,Ticker,Start,End,Downloaded
count,1216,1216,1216,1216
unique,1168,582,575,2
top,COV,1996-01-02,2024-07-08,True
freq,3,487,503,830


TODO:
- methods for downloading historical data and saving it in the cloud
- introduce sector and sub-industry
- find the data for stocks that are delisted
- how to back up the notebook and data into GitHub

Stocks data sources:

1. yfinance - hack-around yahoo finance.
Free but has limitations
https://pypi.org/project/yfinance/

2. Market{Data}:
https://www.marketdata.app

3. Investing.com. Free but need to know the name of the company and proper ticker
https://www.investing.com/equities/